### Goal : Generate the files to measure x264 and x265's performances

##### Libraries

In [1]:
import os, time 
import numpy as np

## A-] Configuration options

We define the configuration options common to x264 and x265, that are influential to study software's performances

In [2]:
# common options and common values, the rest is set to default values
nb_cs = 1

## common options with arg -> replace with "--name-option arg-option"
common_arg = {
              '--aq-strength' : [str(round(i, nb_cs)) for i in np.arange(0, 3.1, 0.5)],
              '--ipratio' : [str(round(i, nb_cs)) for i in np.arange(1, 1.6, 0.1)],
              '--pbratio' : [str(round(i, nb_cs)) for i in np.arange(1, 1.5, 0.1)],
              '--psy-rd' : [str(round(i, nb_cs)) for i in np.arange(0.2, 5.1, 0.2)],
              '--qblur' : [str(round(i, nb_cs)) for i in np.arange(0.2, 0.7, 0.1)],
              '--qcomp' : [str(round(i, nb_cs)) for i in np.arange(0.6, 0.9, 0.1)],
              '--vbv-init' : [str(round(i, nb_cs)) for i in np.arange(0,1,0.1)],
              
              '--aq-mode' : [str(i) for i in range(4)],
              '--b-adapt' : [str(i) for i in range(3)],
              '--bframes' : [str(i) for i in range(0, 17, 2)],
              '--crf' : [str(i) for i in range(0, 52, 5)],
              '--keyint' : [str(i) for i in range(200, 301, 10)],
              '--lookahead-threads' : [str(i) for i in range(5)],
              '--min-keyint' : [str(i) for i in range(20, 31, 1)],
              '--qp' : [str(i) for i in range(0, 51, 10)],
              '--qpstep' : [str(i) for i in range(3, 6, 1)],
              '--qpmin' : [str(i) for i in range(5)],
              '--qpmax' : [str(i) for i in range(60, 70, 1)],
              '--rc-lookahead' : [str(i) for i in range(10, 51, 10)],
              '--ref' : [str(i) for i  in range(1, 7, 1)],
              '--vbv-bufsize' : [str(i) for i in [1000, 2000]],
    
              '--deblock' : ['-2:-2', '-1:-1', '1:1'],
              '--me' : ["dia", "hex", "umh"],
              '--overscan' : ["show", "crop"],
              '--preset' : ["ultrafast", "superfast", "veryfast", "faster", 
                           "fast", "medium", "slow", "slower", "veryslow", 
                           "placebo"],
              '--scenecut' : ['0', '10', '30', '40'],
              '--tune': ['psnr', 'ssim', 'grain', 'animation']
             }

## common options without arg -> replace with "--val-option"
common_val = {
              '--aud' : ["--aud", None],
              '--constrained-intra' : ['--constrained-intra', None],
              '--intra-refresh' : ['--intra-refresh', None],
              '--no-asm' : [None, '--no-asm'],
              '--slow-firstpass' : ['--slow-firstpass', None],
              '--weightb' : ['--weightb', '--no-weightb']
             }

## constraints implies (feature_name1, feature_values1, feature_name2, feature_values2)
constraints =  [
                ('--scenecut', ['10', '30', '40'], '--intra-refresh', None),
                ('--ref', [str(i) for i  in range(2, 7, 1)], '--intra-refresh', None),
               ]

print("We leverage {0} common configuration options between x264 and x265"
      .format(len(common_val)+len(common_arg)))

We leverage 33 common configuration options between x264 and x265


##  B-] Input videos

We apply these configurations on input videos, so we have to define the part of the command line related to the videos

In [3]:
## Group 1

# Gaming_360P-56fe
gaming=" --input-res 640x360 --fps 24 -o test.mp4 ../inputs/original_videos_Gaming_360P_Gaming_360P-56fe.mkv"
# Sports_360P-4545
sports=" --input-res 624x464 --fps 25 -o test.mp4 ../inputs/original_videos_Sports_360P_Sports_360P-4545.mkv"


## Group 2

# Animation_480P-087e
animation=" --input-res 720x480 --fps 15.42 -o test.mp4 ../inputs/original_videos_Animation_480P_Animation_480P-087e.mkv"
# CoverSong_360P-5d20
cover=" --input-res 480x360 --fps 30 -o test.mp4 ../inputs/original_videos_CoverSong_360P_CoverSong_360P-5d20.mkv"


## Group 3

# Lecture_360P-114f
lecture=" --input-res 640x360 --fps 25 -o test.mp4 ../inputs/original_videos_Lecture_360P_Lecture_360P-114f.mkv"
# MusicVideo_360P-5699
music=" --input-res 640x360 --fps 30 -o test.mp4 ../inputs/original_videos_MusicVideo_360P_MusicVideo_360P-5699.mkv"


## Group 4

# LiveMusic_360P-1d94
live=" --input-res 640x360 --fps 25 -o test.mp4 ../inputs/original_videos_LiveMusic_360P_LiveMusic_360P-1d94.mkv"
# LyricVideo_360P-5e87
lyric=" --input-res 640x360 --fps 6 -o test.mp4 ../inputs/original_videos_LyricVideo_360P_LyricVideo_360P-5e87.mkv"

vids = [gaming, sports, animation, cover, lecture, music, live, lyric]

## C-] Software

In [4]:
softs = ['x264 ', 'x265 ']

## D-] Testing these values

>**N.B.** : to automatically test all the possible configurations, we save the logs in the logs.txt file. We then search for "[error]" in the txt file.

##### 1 - Software + Inputs

In [5]:
with open("bash_x26.sh", "w") as f:
    f.write("echo 'START VIDEOS'\n")

for software in softs:
    for video in vids:
        with open("bash_x26.sh", "a+") as f:
            f.write("echo 'testing "+video+"'\n")
            f.write(software+video+"\n")

with open("bash_x26.sh", "a+") as f:
    f.write("echo 'END VIDEOS'\n")

##### 2 - Software + Values (configuration options without argument)

In [6]:
with open("bash_x26.sh", "a+") as f:
    f.write("echo 'START VALUES OPTIONS'\n")

for software in softs:
    for feature_name in common_val.keys():
        values = common_val[feature_name]
        for val in values:
            if val:
                with open("bash_x26.sh", "a+") as f:
                    f.write("echo '"+val+"'\n")
                    f.write(software+val+lyric+"\n")

with open("bash_x26.sh", "a+") as f:
    f.write("echo 'END VALUES OPTIONS'\n")

##### 3 - Software + Arguments (configuration options with argument)

In [7]:
with open("bash_x26.sh", "a+") as f:
    f.write("echo 'START ARGUMENTS OPTIONS'\n")

for software in softs:
    for feature_name in common_arg.keys():
        args = common_arg[feature_name]
        for argument in args:
            with open("bash_x26.sh", "a+") as f:
                f.write("echo '"+feature_name+" "+argument+"'\n")
                f.write(software+feature_name+' '+argument+lyric+"\n")

with open("bash_x26.sh", "a+") as f:
    f.write("echo 'END ARGUMENTS OPTIONS'\n")

>Bash command, **to run in command line once in the folder** : ``bash bash_x26.sh &> logs.txt``

## E-] Generating the configurations

### 1- Generation 

We used random sampling to generate the configurations, w.r.t the constraints.

In [48]:
cmd_line_args = dict()

# values and boolean
for feature_name in common_val:
    values = common_val[feature_name]
    val = values[np.random.randint(len(values))]
    if val:
        cmd_line_args[feature_name] = val

# features with arguments
for feature_name in common_arg:
    args = common_arg[feature_name]
    arg = args[np.random.randint(len(args))]
    cmd_line_args[feature_name] = arg
    
# constraints
for cs in constraints:
    for cmd_name in cmd_line_args:
        cmd = cmd_line_args[cmd_name]
        if cs[0]==cmd_name and cmd in cs[1]:
            if cs[2] in cmd_line_args:
                #print("{0} changed to {1}".format(cs[2],cs[3]))
                cmd_line_args[cs[2]] = cs[3]
                
print(cmd_line_args)

{'--constrained-intra': '--constrained-intra', '--no-asm': '--no-asm', '--slow-firstpass': '--slow-firstpass', '--weightb': '--no-weightb', '--aq-strength': '3.0', '--ipratio': '1.4', '--pbratio': '1.4', '--psy-rd': '4.6', '--qblur': '0.6', '--qcomp': '0.6', '--vbv-init': '0.9', '--aq-mode': '2', '--b-adapt': '1', '--bframes': '6', '--crf': '20', '--keyint': '220', '--lookahead-threads': '2', '--min-keyint': '23', '--qp': '30', '--qpstep': '3', '--qpmin': '1', '--qpmax': '61', '--rc-lookahead': '10', '--ref': '4', '--vbv-bufsize': '2000', '--deblock': '1:1', '--me': 'hex', '--overscan': 'crop', '--preset': 'superfast', '--scenecut': '30', '--tune': 'animation'}


### 2- Infer constraints from logs

Loop -> test new configs -> find new constraints -> add constraints to the generation process -> test new configs w.r.t. the constraints -> find new constraints -> etc.